In [ ]:
!pip install datasets

In [10]:
from datasets import load_dataset

In [11]:
def ner_tags_to_spans(samples, tag_to_id):
    """
    Converts NER tags in the dataset samples to spans (start, end, entity type).
    
    Args:
        samples (dict): A dictionary containing the tokens and NER tags.
        tag_to_id (dict): A dictionary mapping NER tags to IDs.
    
    Returns:
        dict: A dictionary containing tokenized text and corresponding NER spans.
    """
    ner_tags = samples["ner_tags"]
    id_to_tag = {v: k for k, v in tag_to_id.items()}
    spans = []
    start_pos = None
    entity_name = None

    for i, tag in enumerate(ner_tags):
        if tag == 0:  # 'O' tag
            if entity_name is not None:
                spans.append((start_pos, i - 1, entity_name))
                entity_name = None
                start_pos = None
        else:
            tag_name = id_to_tag[tag]
            if tag_name.startswith('B-'):
                if entity_name is not None:
                    spans.append((start_pos, i - 1, entity_name))
                entity_name = tag_name[2:]
                start_pos = i
            elif tag_name.startswith('I-'):
                continue

    # Handle the last entity if the sentence ends with an entity
    if entity_name is not None:
        spans.append((start_pos, len(samples["tokens"]) - 1, entity_name))
    
    return {"tokenized_text": samples["tokens"], "ner": spans}

In [ ]:
# step 1: load data
dataset = load_dataset("eriktks/conll2003")

In [13]:
# Step 2: Define NER tag-to-ID mapping
tag_to_id = {
    'O': 0, 'B-person': 1, 'I-person': 2, 'B-organization': 3, 'I-organization': 4,
    'B-location': 5, 'I-location': 6, 'B-others': 7, 'I-others': 8
}

In [14]:
# Convert NER tags to spans for the training data
gliner_data_conll = [ner_tags_to_spans(i, tag_to_id) for i in dataset['train']]

In [15]:
# Load the pre-trained GLiNER model
from gliner import GLiNER
import torch

model = GLiNER.from_pretrained("urchade/gliner_small", load_tokenizer=True) #true if a model was trained from scratch with new code base

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = model.to(device)

In [17]:
# Evaluate the model on the first 100 samples
evaluation_results = model.evaluate(
    gliner_data_conll[:100], flat_ner=True, entity_types=["person", "organization", "location", "others"]
)

In [18]:
print(evaluation_results)

('P: 63.13%\tR: 71.43%\tF1: 67.02%\n', 0.6702412868632708)
